In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

%matplotlib inline
CLASSIFICATIONS=[
    ('../results/classification_submissions/v37.csv', 0.85424),
    ('../results/classification_submissions/v4.csv',  0.85674),
    ('../results/classification_submissions/v3.csv',  0.85341),
    ('../results/classification_submissions/v1.csv',  0.56399),
    ('../results/classification_submissions/v34.csv', 0.84716),
    ('../results/classification_submissions/v33.csv', 0.84314),
    ('../results/classification_submissions/0.85188.csv', 0.85188),
    ('../results/classification_submissions/0.85383.csv', 0.85383),
]

In [13]:
def convert_lb_to_accuracy(c, lb):
    positive_predictions = (c['EncodedPixels'] != '').astype(np.int8).values
    negative_predictions = 1 - positive_predictions
    
    false_positives = (0.85674 - lb) / 0.0001387
    true_positives = positive_predictions.sum() - false_positives
    
    missing_positives = 1032 - true_positives

    return true_positives / positive_predictions.sum(), missing_positives / negative_predictions.sum()

def clean_classifications(i, c, lb):
    c[['EncodedPixels']] = c[['EncodedPixels']].fillna(value='')
    c['class'] = c['ImageId_ClassId'].str[-1].astype(np.int8)
    c['image_id'] = c['ImageId_ClassId'].str.split('_', expand=True)[0]
    prediction_acc, miss_acc = convert_lb_to_accuracy(c, lb)
    
    proba_col = 'proba_{}'.format(i)
    c[proba_col] = miss_acc
    c.loc[c['EncodedPixels'] !='',proba_col] = prediction_acc
    return c[['image_id', 'class', proba_col]]

results = [clean_classifications(i, pd.read_csv(r[0]), r[1]) for i, r in enumerate(CLASSIFICATIONS)]

In [20]:
from functools import reduce

res = reduce(lambda l,r: pd.merge(l, r, on=['image_id', 'class'], how='inner'), results)
res['defect_proba'] = res['proba_0'] * res['proba_1'] * res['proba_2'] * res['proba_3'] * res['proba_4']* res['proba_5']* res['proba_6']* res['proba_7']
res['defect_proba'] = res['defect_proba'] / res['defect_proba'].max()
res['nondefect_proba'] = 1 - res['defect_proba']

In [21]:
res.sort_values('nondefect_proba')

,image_id,class,proba_0,proba_1,proba_2,proba_3,proba_4,proba_5,proba_6,proba_7,defect_proba,nondefect_proba
3054,de5881c2b.jpg,3,0.935856,0.1363,0.1399,0.197366,0.868938,0.837660,0.890501,0.943601,1.000000,0.000000
2822,b0b5a3357.jpg,3,0.935856,0.1363,0.1399,0.197366,0.868938,0.837660,0.890501,0.943601,1.000000,0.000000
4310,b6d419268.jpg,3,0.935856,0.1363,0.1399,0.197366,0.868938,0.837660,0.890501,0.943601,1.000000,0.000000
3318,e968f8497.jpg,3,0.935856,0.1363,0.1399,0.197366,0.868938,0.837660,0.890501,0.943601,1.000000,0.000000
918,59c0a3475.jpg,3,0.935856,0.1363,0.1399,0.197366,0.868938,0.837660,0.890501,0.943601,1.000000,0.000000
6578,33d296fef.jpg,3,0.935856,0.1363,0.1399,0.197366,0.868938,0.837660,0.890501,0.943601,1.000000,0.000000
1086,f83e863db.jpg,3,0.935856,0.1363,0.1399,0.197366,0.868938,0.837660,0.890501,0.943601,1.000000,0.000000
6990,9add1b7db.jpg,3,0.935856,0.1363,0.1399,0.197366,0.868938,0.837660,0.890501,0.943601,1.000000,0.000000
150,cc6e5e804.jpg,3,0.935856,0.1363,0.1399,0.197366,0.868938,0.837660,0.890501,0.943601,1.000000,0.000000
3646,256aeb46a.jpg,3,0.935856,0.1363,0.1399,0.197366,0.868938,0.837660,0.890501,0.943601,1.000000,0.000000


In [31]:
res[res['defect_proba'] > 0.9999]

,image_id,class,proba_0,proba_1,proba_2,proba_3,proba_4,proba_5,proba_6,proba_7,defect_proba,nondefect_proba
62,d670249ed.jpg,3,0.935856,0.1363,0.1399,0.197366,0.868938,0.83766,0.890501,0.943601,1.0,0.0
94,cd17f385a.jpg,3,0.935856,0.1363,0.1399,0.197366,0.868938,0.83766,0.890501,0.943601,1.0,0.0
110,165be877c.jpg,3,0.935856,0.1363,0.1399,0.197366,0.868938,0.83766,0.890501,0.943601,1.0,0.0
130,cb241fa5d.jpg,3,0.935856,0.1363,0.1399,0.197366,0.868938,0.83766,0.890501,0.943601,1.0,0.0
150,cc6e5e804.jpg,3,0.935856,0.1363,0.1399,0.197366,0.868938,0.83766,0.890501,0.943601,1.0,0.0
158,b3912fc50.jpg,3,0.935856,0.1363,0.1399,0.197366,0.868938,0.83766,0.890501,0.943601,1.0,0.0
182,541e5bfec.jpg,3,0.935856,0.1363,0.1399,0.197366,0.868938,0.83766,0.890501,0.943601,1.0,0.0
226,1bbf4b4c0.jpg,3,0.935856,0.1363,0.1399,0.197366,0.868938,0.83766,0.890501,0.943601,1.0,0.0
254,1d617527f.jpg,3,0.935856,0.1363,0.1399,0.197366,0.868938,0.83766,0.890501,0.943601,1.0,0.0
294,f947a5bf6.jpg,3,0.935856,0.1363,0.1399,0.197366,0.868938,0.83766,0.890501,0.943601,1.0,0.0


In [43]:
convert_lb_to_accuracy(res1, 0.84716)

0.7541995191752618